In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import requests

In [4]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text


chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)


max_length = 40  # Length of input sequences
step = 3  # Step size for sampling sequences
sequences = []
next_chars = []

for i in range(0, len(text) - max_length, step):
    sequences.append(text[i:i + max_length])
    next_chars.append(text[i + max_length])

# Vectorize the sequences
X = np.zeros((len(sequences), max_length, vocab_size), dtype=bool)
y = np.zeros((len(sequences), vocab_size), dtype=bool)

for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1

In [5]:
model = models.Sequential([
    layers.LSTM(128, input_shape=(max_length, vocab_size)),
    layers.Dense(vocab_size, activation='softmax')
])

# Summary of the model
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          99,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 65)                  │           8,385 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 107,713 (420.75 KB)

 Trainable params: 107,713 (420.75 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy')

In [8]:
history = model.fit(X, y,
                    epochs=20,  # More epochs for better results (e.g., 50+)
                    batch_size=128,
                    verbose=1,)

Epoch 1/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 2.7692
Epoch 2/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 2.0648
Epoch 3/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.9126
Epoch 4/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.8267
Epoch 5/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.7520
Epoch 6/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.7014
Epoch 7/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.6618
Epoch 8/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.6326
Epoch 9/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.6018
Epoch 10/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.5827
Epoch 11/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.5597
Epoch 12/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.5419
Epoch 13/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.5189
Epoch 14/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.5089
Epoch 15/20
290

In [11]:
def generate_text(model, seed_text, length=200, temperature=1.0):
    generated = seed_text
    for _ in range(length):
        # Prepare the input
        x_pred = np.zeros((1, max_length, vocab_size))
        for t, char in enumerate(seed_text[-max_length:]):
            x_pred[0, t, char_to_idx[char]] = 1

        # Predict the next character
        preds = model.predict(x_pred, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds + 1e-10) / temperature  # Apply temperature for diversity
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        # Sample the next character
        next_idx = np.random.choice(range(vocab_size), p=preds)
        next_char = idx_to_char[next_idx]

        # Append and update seed
        generated += next_char
        seed_text = seed_text[1:] + next_char

    return generated

# Generate some text after training
seed_text = "To be or not to be"
generated_text = generate_text(model, seed_text, length=100, temperature=0.6)
print("Generated Text:\n", generated_text)

Generated Text:
 To be or not to be
ib''loa'srt!'alslslsllalrla,.a,?lllllt!..:rdddddddl
s'a,sllrl,lltrrllsll;.rslsll,alrassl
,sll',lbss
